In [1]:
using Rocket
using ReactiveMP
using GraphPPL
using BenchmarkTools
using Distributions
using MacroTools

┌ Info: Precompiling ReactiveMP [a194aa59-28ba-4574-a09c-4a745416d6e3]
└ @ Base loading.jl:1278
┌ Warning: Package ReactiveMP does not have Rocket in its dependencies:
│ - If you have ReactiveMP checked out for development and have
│   added Rocket as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with ReactiveMP
└ Loading Rocket into ReactiveMP from project dependency, future warnings for ReactiveMP are suppressed.
┌ Info: Precompiling GraphPPL [b3f8163a-e979-4e85-b43e-1f63d8c8b42c]
└ @ Base loading.jl:1278
┌ Warning: Package GraphPPL does not have ReactiveMP in its dependencies:
│ - If you have GraphPPL checked out for development and have
│   added ReactiveMP as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with GraphPPL
└ Loading ReactiveMP into GraphPPL from project dependency, future 

In [2]:
@model function smoothing(n, x0, c::ConstVariable, P::ConstVariable)
    
    x_prior ~ NormalMeanVariance(mean(x0), cov(x0)) 

    x = randomvar(n)
    y = datavar(Float64, n)

    x_prev = x_prior

    for i in 1:n
        x[i] ~ x_prev + c
        y[i] ~ NormalMeanVariance(x[i], P)
        
        x_prev = x[i]
    end

    return x, y
end

smoothing (generic function with 1 method)

In [3]:
using Random

In [4]:
P = 1.0

Random.seed!(123)

n = 250
data = -5.0 .+ collect(1:n) + rand(Normal(0.0, sqrt(P)), n);

In [5]:
function inference(data, x0, c, P)
    n = length(data)
    
    model, (x, y) = smoothing(n, x0, c, P);

    ms_buffer = Vector{Marginal}(undef, n)
    fe_buffer = nothing
    
    marginals = collectLatest(getmarginals(x))
    
    fe_subscription = subscribe!(score(BetheFreeEnergy(), model, AsapScheduler()), (fe) -> fe_buffer = fe)
    ms_subscription = subscribe!(marginals, (ms) -> copyto!(ms_buffer, ms))
    
    update!(y, data)
    
    unsubscribe!(ms_subscription)
    unsubscribe!(fe_subscription)
    
    return ms_buffer, fe_buffer
end

inference (generic function with 1 method)

In [6]:
# c[1] is C
# c[2] is μ0
function f(c)
    x0_prior = NormalMeanVariance(c[2], 100.0)
    ms, fe = inference(data, x0_prior, c[1], P)
    return fe
end

f (generic function with 1 method)

In [7]:
using Optim

In [8]:
res = optimize(f, ones(2), GradientDescent(), Optim.Options(g_tol = 1e-3, iterations = 100, store_trace = true, show_trace = true))

Iter     Function value   Gradient norm 
     0     3.655789e+02     8.149754e+02
 * time: 0.0038061141967773438
     1     3.653239e+02     5.997076e-02
 * time: 0.6496071815490723
     2     3.652238e+02     3.400217e+02
 * time: 2.530269145965576
     3     3.651794e+02     2.660345e-02
 * time: 2.7750680446624756
     4     3.651615e+02     1.517870e+02
 * time: 4.507191181182861
     5     3.651526e+02     1.309935e-02
 * time: 4.758658170700073
     6     3.651483e+02     7.474478e+01
 * time: 6.4789040088653564
     7     3.651461e+02     6.495536e-03
 * time: 6.7258830070495605
     8     3.651450e+02     3.697864e+01
 * time: 8.428382158279419
     9     3.651445e+02     3.026425e-03
 * time: 8.675896167755127
    10     3.651443e+02     1.650103e+01
 * time: 10.253507137298584
    11     3.651442e+02     1.959583e-03
 * time: 10.501447200775146
    12     3.651442e+02     8.163634e+00
 * time: 12.089855194091797
    13     3.651442e+02     1.649326e-03
 * time: 12.32220911979

 * Status: success

 * Candidate solution
    Final objective value:     3.651440e+02

 * Found with
    Algorithm:     Gradient Descent

 * Convergence measures
    |x - x'|               = 3.15e-06 ≰ 0.0e+00
    |x - x'|/|x'|          = 6.32e-07 ≰ 0.0e+00
    |f(x) - f(x')|         = 6.45e-06 ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 1.77e-08 ≰ 0.0e+00
    |g(x)|                 = 1.44e-04 ≤ 1.0e-03

 * Work counters
    Seconds run:   16  (vs limit Inf)
    Iterations:    17
    f(x) calls:    128
    ∇f(x) calls:   128


In [9]:
res.minimizer # Real values are indeed (c = 1.0 and μ0 = -5.0)

2-element Array{Float64,1}:
  1.0006316018959704
 -4.983657748477323